# wordslab-notebooks-lib.notebook

> Access wordslab-notebooks Jupyterlab extension version, current notebook path, json content and cell id, and create or update cells.

In [ ]:
#| default_exp notebook

In [ ]:
#| export
from ast import literal_eval
import asyncio
from datetime import datetime
from functools import partial
from html import escape
from inspect import currentframe, getattr_static, getdoc, isfunction, ismethod, signature
import re
from textwrap import dedent
from types import ModuleType, FunctionType, MethodType, BuiltinFunctionType
import time
import typing
from typing import Optional, Union, Literal

from IPython.core.getipython import get_ipython
from IPython.core.interactiveshell import InteractiveShell
from IPython.core.oinspect import Inspector
from IPython.display import display, HTML, Markdown, clear_output
from comm import create_comm
import nbformat

from fastcore.utils import L, patch
from fastcore.xml import to_xml, Note, Prompt, Code, Source, Outputs, User, Assistant, Out, Var
from toolslm.funccall import get_schema

from wordslab_notebooks_lib.env import WordslabEnv
from wordslab_notebooks_lib.chat import OllamaModelClient, OpenRouterModelClient, Tools

## Work with AI in a Jupyterlab notebook - the Solveit method

A Jupyter notebook is a convenient way to build context for a LLM one cell after the other: you are working in a fully editable conversation, while interacting with AI and with code.

**Jeremy Howard** and his team at **Answer.ai** explored how to work efficiently in this kind of conversation: they developed a method and platform called Solveit.

https://solve.it.com/

We would like to replicate this approach to working with AI in a wordslab notebooks environment. 

Here is how we chose to do it:
- in a jupyterlab notebook, there are two types of cells: markdown and code
- we want to simulate a third type of cell: a "prompt" cell
- the content of this cell is a prompt (text in markdown format) which is sent to an llm when the cell is executed, along with the text of all the cells situated above in the notebook (context)
- the llm response is streamed just below and formatted as markdown.

To simulate this "prompt" cell we need to develop a **Jupyterlab frontend extension** which implements the following behaviors :
- three buttons are added to the cell toolbar: "note", "prompt", "code"
- a click on one of these buttons changes the type of the cell
  - "note" selects a classic markdown cell
  - "prompt" selects a code cell, modified with the special "prompt behavior" defined below
  - "code" selects a classic code cell
- a "prompt" cell is distinguished from a regular code cell by a metadata property registered in the ipynb file
- each cell type is visualized by a specific color in the left border of the cell
  - "note" cell has a green border
  - "prompt" cell a red border
  - "code" cell has a blue border
- the "prompt" cell is a code cell with the specific modified behaviors
  - the code syntax highlighting is replaced by markdown syntax highlighting when the user types text in this cell
  - when the user executes this cell, the frontend extension does the following
    - calls the kernel to inject the following variables
      - __notebook_path with the path and name of the notebook in the workspace
      - __notebook_content with the full json representation of the notebook
      - __cell_id with the id of the current cell
    - then calls the kernel to execute a specific chat(message) python function
      - where the message parameter is the content of the cell
      - and the content of the notebook above the current cell is inluded as context
  - the python chat() function streams the response tokens from the llm to the output section of the code cell, with markdown rendering

See the section "Develop a Jupyterlab frontend extension" at the bottom of this page to understand how the extension was developed.

## Install the Jupyterlab extension - wordslab-notebooks-lib

If you want to use "prompt" cells, you will first need to install the Jupyterlab frontend extension:
- activate your Jupyterlab python virtual environment
- **pip install wordslab-notebooks-lib**
- restart your Jupyterlab server

The extension is **already pre-installed in the wordslab-notebooks environment**.

To be clear: the wordslab-notebooks-lib package contains both: the Javascript Jupyterlab frontend extension AND the python library wich is loaded in the python kernel.

The Jupyterlab frontend extension is reloaded and re-initialized each time you refresh your browser page: 
- to check is the extension is installed and running, look at the browser console and llok for the message 'Wordslab notebooks extension vx.y.z activated'
- hit the refresh button if you encounter a bug and the extension stops working

## Extend the ipython kernel with useful utilities

You also need to install the wordslab-notebooks-lib library in the virtual environnement used by the ipython kernel which runs each notebook in which you want to use the Solveit method.

It is the client of the Jupyterlab extension, and provides many utilties and tools which support this new way of working with AI.

The main python object used to interact with an ipython kernel is the InteractiveShell. You get an instance of it with the get_ipython() method.

In [ ]:
shell = get_ipython()
type(shell)

ipykernel.zmqshell.ZMQInteractiveShell

We will start by extending this shell with capabilities useful to work in a notebook with the Solveit method. These extensions are inspired by the library **ipykernel_helper** from **Answer.ai**. As of december 2025, this library is not open source, but it is available to users in the solve.it.com environment and is a dependency of other Apache 2.0 libraries, so I think it is OK to use it as an inspiration.

In [ ]:
#| export
@patch
def _get_info(self: Inspector, obj, oname='', formatter=None, info=None, detail_level=0, omit_sections=()):
    """Custom Python formatter for ?? output:
    - suppress the useless code indentation
    - wrap the code in a markdown python code block
    - display the output as markdown
    """
    orig = self._orig__get_info(obj, oname=oname, formatter=formatter, info=info,
                                detail_level=detail_level, omit_sections=omit_sections)
    if detail_level == 0:
        return orig
    info_dict = self.info(obj, oname=oname, info=info, detail_level=detail_level)
    out = []
    if c := info_dict.get('source'): 
        out.append(f"\n```python\n{dedent(c)}\n```")
    if c := info_dict.get('file'): 
        out.append(f"**File:** `{c}`")
    return {'text/markdown': '\n\n'.join(out), 'text/html': '', 'text/plain': orig['text/plain']}

In [ ]:
escape??


```python
def escape(s, quote=True):
    """
    Replace special characters "&", "<" and ">" to HTML-safe sequences.
    If the optional flag quote is true (the default), the quotation mark
    characters, both double quote (") and single quote (') characters are also
    translated.
    """
    s = s.replace("&", "&amp;") # Must be done first!
    s = s.replace("<", "&lt;")
    s = s.replace(">", "&gt;")
    if quote:
        s = s.replace('"', "&quot;")
        s = s.replace('\'', "&#x27;")
    return s
```

**File:** `/home/python/cpython-3.12.12-linux-x86_64-gnu/lib/python3.12/html/__init__.py`

In [ ]:
#| export
def _safe_str(obj, max_str_len=200):
    "Safely get the string representation of an object, truncating if it exceeds max_len."
    try:
        s = str(obj)
        return s[:max_str_len] + ("…" if len(s) > max_str_len else "")
    except Exception as e: 
        return f"<str error: {str(e)}>"


@patch
def user_items(self: InteractiveShell, max_str_len=200, xtra_ignore=()):
    """Get an overview of the variables & functions defined by the user so far in the notebook.
    The value addded by this function is to filter out all internal ipython and wordslab variables.
    Returns a tuple of dictionaries (user_variables, user_functions):
    - the keys are the variables or function names
    - the value is a truncated string representation of the variable value or the function signature
    The `max_str_len` parameter is used to truncate the string representation of the variables.
    The `xtra_ignore` parameter is used to hide additional names from the result. 
    """
    ns, nsh = self.user_ns, self.user_ns_hidden
    ignore = set()  # Add here the wordslab specific vars and funcs we want to hide
    ignore.add(xtra_ignore)
    rm_types = (
        type, FunctionType, ModuleType, MethodType, BuiltinFunctionType,
        getattr(typing, '_SpecialGenericAlias', ()),
        getattr(typing, '_GenericAlias', ()),
        getattr(typing, '_SpecialForm', ())
    )
    user_items = {k: v for k, v in ns.items()
                  if k not in ignore and k not in nsh}
    user_vars = {k: _safe_str(v, max_str_len=max_str_len)
                 for k, v in user_items.items() if not k.startswith('_') and not isinstance(v, rm_types)}
    user_fns = {k: str(signature(v)) for k, v in user_items.items()
                if isinstance(v, FunctionType) and v.__module__ == '__main__' and not k.startswith('__')}
    return user_vars, user_fns

In [ ]:
variables, functions = shell.user_items()
variables, functions

({'Code': "functools.partial(<function ft>, 'code', void_=False)",
  'Source': "functools.partial(<function ft>, 'source', void_=True)",
  'shell': '<ipykernel.zmqshell.ZMQInteractiveShell object>',
  'user_items': 'None'},
 {'_safe_str': '(obj, max_str_len=200)'})

In [ ]:
#| export
@patch
def get_variables_values(self: InteractiveShell, var_names: list, literal=True):
    """Get a safe and serializable representation of variables values from the user namespace.
    This method preserves real Python values when they are safe literals, otherwise it falls back to strings.
    You can call it in two modes:
    - literal = True : Preserve actual Python values when safe, best for internal tools
    - literal = False : Force everything to strings, best for logging / UI display / debug output
    """
    ns = self.user_ns

    def _maybe_eval(o):
        try:
            literal_eval(repr(o))
            return o
        except:
            return str(o)
    return {v: _maybe_eval(ns[v]) if literal else str(ns[v]) for v in var_names if v in ns}

In [ ]:
shell.get_variables_values(var_names=["variables", "functions"])

{'variables': {'Code': "functools.partial(<function ft>, 'code', void_=False)",
  'Source': "functools.partial(<function ft>, 'source', void_=True)",
  'shell': '<ipykernel.zmqshell.ZMQInteractiveShell object>',
  'user_items': 'None'},
 'functions': {'_safe_str': '(obj, max_str_len=200)'}}

In [ ]:
#| export
def _get_schema(ns: dict, t):
    "Check if tool `t` has errors."
    if t not in ns:
        return f"`{t}` not found. Did you run the cell where it is defined?"
    try:
        return {'type': 'function', 'function': get_schema(ns[t], pname='parameters')}
    except Exception as e:
        return f"`{t}`: {e}."

@patch
def get_tools_schemas_and_functions(self: InteractiveShell, func_names: list):
    """Get a json schema and a function object for the functions defined in the user namespace which can be used as tools."""
    ns = self.user_ns
    return {f: (_get_schema(ns, f), ns[f]) for f in func_names if f in ns}

In [ ]:
# Example tool definition
def example_sum(
    a: int,  # First thing to sum
    b: int = 1,  # Second thing to sum
) -> int:  # The sum of the inputs
    "Adds a + b."
    return a + b

In [ ]:
shell.get_tools_schemas_and_functions(["example_sum"])

{'example_sum': ({'type': 'function',
   'function': {'name': 'example_sum',
    'description': 'Adds a + b.\n\nReturns:\n- type: integer',
    'parameters': {'type': 'object',
     'properties': {'a': {'type': 'integer',
       'description': 'First thing to sum'},
      'b': {'type': 'integer',
       'description': 'Second thing to sum',
       'default': 1}},
     'required': ['a']}}},
  <function __main__.example_sum(a: int, b: int = 1) -> int>)}

## Access the notebook path, cells content and current cell id

The 4 notebook properties below are **silently injected by the Jupyterlab frontend extension** before each code cell is executed.

This will not work if the wordslab-notebooks-lib Jupyterlab extension is not installed.

In [ ]:
#| export
def _find_frame_dict(var: str):
    "Find the dict (globals or locals) containing var"
    frame = currentframe().f_back.f_back
    while frame:
        if var in frame.f_globals:
            return frame.f_globals
        frame = frame.f_back
    raise ValueError(f"Could not find {var} in any scope")

def find_var(var: str):
    "Search for var in all frames of the call stack"
    return _find_frame_dict(var)[var]

In [ ]:
#| export
class WordslabNotebook:
    """Jupyterlab notebook introspection and metaprogramming."""

    def __init__(self):
        # Check if we are running inside a Jupyter notebook
        if get_ipython() is None:
            raise RuntimeError("This class can only be used in the context of a Jupyter notebook")

        # Check if the wordslab-notebooks-lib Jupyterlab frontend extension is installed
        try:
            find_var("__wordslab_extension_version")
            self.jupyterlab_extension_installed = True
        except:
            self.jupyterlab_extension_installed = False

        # Initialize a communication channel with the frontend extension
        if self.jupyterlab_extension_installed:
            self._comm = WordslabNotebook.JupyterlabExtensionComm()

    def _ensure_jupyterlab_extension(self):
        if not self.jupyterlab_extension_installed:
            raise RuntimeError(
                "The JupyterLab extension for wordslab-notebooks is not activated: "
                "please execute `pip install wordslab-notebooks-lib` in JupyterLab virtual environment "
                "and refresh your web browser."
            )

    # --------------------------------
    # Frontend -> Kernel communication
    # --------------------------------

    # The frontend extension injects the following variables before each cell execution

    @property
    def jupyterlab_extension_version(self):
        """wordslab-notebooks-lib version number injected by the Jupyterlab frontend extension"""
        self._ensure_jupyterlab_extension()
        return find_var("__wordslab_extension_version")

    @property
    def path(self):
        """Relative path of the notebook .ipynb file in the notebook workspace"""
        self._ensure_jupyterlab_extension()
        return find_var("__notebook_path")

    @property
    def content(self):
        """Full content of the notebook returned as a NotebookNode object from the nbformat library"""
        self._ensure_jupyterlab_extension()
        return nbformat.from_dict(find_var("__notebook_content"))

    @property
    def cell_id(self):
        """Unique ID of the current notebook cell, useful to locate the current cell in the full notebook content"""
        self._ensure_jupyterlab_extension()
        return find_var("__cell_id")

    # --------------------------------
    # Kernel -> Frontend communication
    # --------------------------------

    # The kernel sends commands to the frontend extension through this comms channel

    class JupyterlabExtensionComm:
        def __init__(self, target_name='wordslab_notebooks', timeout=2.0):
            self.target_name = target_name
            self._init_comm()
            self.timeout = timeout
            self.result = None

        def _init_comm(self):
            self.comm = create_comm(target_name=self.target_name)
            self.comm.on_msg(self._on_msg)

        def _on_msg(self, msg):
            self.result = msg['content']['data']

        async def send(self, data):
            self.result = None
            self.comm.send(data)

            start = time.time()
            while self.result is None and (time.time() - start) < self.timeout:
                await asyncio.sleep(0.01)

            if self.result is None:
                self._init_comm()
                raise TimeoutError('No response from Jupyterlab frontend. If you just refreshed the browser, this timeout is expected: please retry running this cell.')

            return self.result

In [ ]:
notebook = WordslabNotebook()

In [ ]:
notebook.jupyterlab_extension_version

'0.0.13'

In [ ]:
notebook.path

'wordslab-notebooks-lib/nbs/04_notebook.ipynb'

In [ ]:
notebook.content.metadata

{'kernelspec': {'display_name': 'wordslab-notebooks-lib',
  'language': 'python',
  'name': 'wordslab-notebooks-lib'},
 'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
  'file_extension': '.py',
  'mimetype': 'text/x-python',
  'name': 'python',
  'nbconvert_exporter': 'python',
  'pygments_lexer': 'ipython3',
  'version': '3.12.12'}}

In [ ]:
notebook.cell_id

'd16ad869-d651-40bd-af2c-623d82b4edf0'

In [ ]:
notebook.cell_id

'59347d6e-d1f7-4d23-b041-143e42887f6d'

## Create, update, delete and run notebook cells

These methods can be used to manipulate the notebook cells when the wordslab-notebooks-lib Jupyterlab frontend extension is installed.

They are inspired by the library **dialoghelper** from **Answer.ai**, but are adapted to the standard Jupyterlab context.

In [ ]:
#| export
@patch
async def add_cell(
    self: WordslabNotebook,
    content: str,  # Content of the cell (i.e the prompt, code, or note cell text)
    placement: str = 'add_after',  # Can be 'add_after', 'add_before', 'at_start', 'at_end'
    cell_id: str = None,  # id of the cell that placement is relative to (if None, uses current cell)
    cell_type: str = 'note',  # Cell type, can be 'code', 'note', or 'prompt'
    notebook_path: str = ''  # Notebook to update, defaults to current notebook
):
    """Add a cell to the current notebook or any other opened notebook (`notebook_path`),
    at the start/end of the notebook or before/after any cell (`placement`and `cell_id`),
    with a `cell_type` (note|prompt|code) and `content` (text).
    Returns the new cell id."""
    self._ensure_jupyterlab_extension()
    if notebook_path and notebook_path != self.path:
        if placement not in ('at_start', 'at_end') and not cell_id:
            raise ValueError("`cell_id` or `placement='at_end'`/`placement='at_start'` must be provided when target notebook is different")
    if placement not in ('at_start', 'at_end') and not cell_id:
        cell_id = self.cell_id
    result = await self._comm.send({'action': 'create_cell', 'cell_type': cell_type, 'content': content, 'placement': placement, 'cell_id': cell_id, 'notebook_path': notebook_path})
    if 'success' in result and result['success']:
        return result['cell_id']
    elif 'error' in result:
        raise RuntimeError(result['error'])

In [ ]:
await notebook.add_cell("Test note")

'a1cc961e-e918-4ac7-9398-7c6fa574f6f2'

Test note

In [ ]:
await notebook.add_cell("Test note 2", placement="add_after")

'984dea0c-450c-488f-8115-82e92d56c0ea'

Test note 2

Test note 3

In [ ]:
await notebook.add_cell("Test note 3", placement="add_before")

'1e24f157-66dc-4200-8b12-523c92bb447d'

In [ ]:
await notebook.add_cell("Test note 4", placement="at_start")

'01d8ebd2-7a4d-479a-939a-d345a9660211'

In [ ]:
await notebook.add_cell("Test note 5", placement="at_end")

'b135d91b-63c0-4a73-8172-d7e73e3a5cf3'

In [ ]:
# Creates a new cell at the end of the notebook
await notebook.add_cell("Test note somewhere", placement="somewhere")

'28690fe7-bf10-4781-8067-432f45d18c7f'

In [ ]:
await notebook.add_cell("Test prompt", cell_type='prompt')

'c41b364b-74da-4886-a974-1683e8143864'

In [ ]:
Test prompt

In [ ]:
await notebook.add_cell("Test code", cell_type='code')

'cd02e818-4989-40a8-90ce-8e92a99bcd59'

In [ ]:
Test code

In [ ]:
# Create a new cell of type note
await notebook.add_cell("Test sometype", cell_type='sometype')

'e80dbc6b-0832-423f-8e33-aacc0ea52039'

Test sometype

In [ ]:
await notebook.add_cell("Test bad cell id", cell_id="bad_cell_id")

RuntimeError: Cell not found: bad_cell_id

In [ ]:
await notebook.add_cell("Test other notebook (bad notebook name)", placement="at_start", notebook_path="wordslab-notebooks-lib/nbs/temp.ipynb")

RuntimeError: Notebook not found: wordslab-notebooks-lib/nbs/temp.ipynb. Make sure the notebook is opened in Jupyterlab.

In [ ]:
await notebook.add_cell("Test other notebook (open)", placement="at_start", notebook_path="wordslab-notebooks-lib/nbs/test.ipynb")

'637eb2a9-421c-4a7c-99aa-e2426b9eab64'

In [ ]:
await notebook.add_cell("Test other notebook (closed)", placement="at_start", notebook_path="wordslab-notebooks-lib/nbs/01_env.ipynb")

RuntimeError: Notebook not found: wordslab-notebooks-lib/nbs/01_env.ipynb. Make sure the notebook is opened in Jupyterlab.

In [ ]:
#| export
@patch
async def update_cell(
    self: WordslabNotebook,
    cell_id: str = None,  # id of the cell to update (if None, uses current cell)
    content: str = None,  # Content of the cell (i.e the prompt, code, or note cell text)
    notebook_path: str = ''  # Notebook to update, defaults to current notebook
):
    """Update the cell identified by `cell_id`,
    in the current notebook or any other opened notebook (`notebook_path`),
    with a new `content`.
    Returns the updated cell id."""
    self._ensure_jupyterlab_extension()
    if not cell_id:
        raise ValueError("`cell_id` parameter is mandatory")
    result = await self._comm.send({'action': 'update_cell', 'cell_id': cell_id, 'content': content, 'notebook_path': notebook_path})
    if 'success' in result and result['success']:
        return result['cell_id']
    elif 'error' in result:
        raise RuntimeError(result['error'])

In [ ]:
orig_cell_id = await notebook.add_cell("original cell content")

updated cell content

In [ ]:
await notebook.update_cell(cell_id=orig_cell_id, content="updated cell content")

'293c77e8-5449-407a-8b4f-0d628c45bfc7'

In [ ]:
await notebook.update_cell(content="no id")

ValueError: `cell_id` parameter is mandatory

In [ ]:
await notebook.update_cell(cell_id="bad_cell_id", content="bad id")

RuntimeError: Cell not found: bad_cell_id

In [ ]:
orig_cell_id = await notebook.add_cell("original cell content in other notebook", placement="at_start", notebook_path="wordslab-notebooks-lib/nbs/test.ipynb")

In [ ]:
await notebook.update_cell(cell_id=orig_cell_id, content="updated cell content in other notebook", notebook_path="wordslab-notebooks-lib/nbs/test.ipynb")

'55d9b34d-7e39-4cb6-9c95-eb0b9e0176f3'

In [ ]:
#| export
@patch
async def delete_cell(
    self: WordslabNotebook,
    cell_id: str = None,  # id of cell to delete
    notebook_path: str = ''  # Notebook to update, defaults to current notebook
):
    """"Update the cell identified by `cell_id`,
    in the current notebook or any other opened notebook (`notebook_path`).
    Returns the deleted cell id."""
    self._ensure_jupyterlab_extension()
    if not cell_id:
        raise ValueError("`cell_id` parameter is mandatory")
    result = await self._comm.send({'action': 'delete_cell', 'cell_id': cell_id, 'notebook_path': notebook_path})
    if 'success' in result and result['success']:
        return result['cell_id']
    elif 'error' in result:
        raise RuntimeError(result['error'])

In [ ]:
await notebook.delete_cell(cell_id="01d8ebd2-7a4d-479a-939a-d345a9660211")

'01d8ebd2-7a4d-479a-939a-d345a9660211'

In [ ]:
await notebook.delete_cell(cell_id="55d9b34d-7e39-4cb6-9c95-eb0b9e0176f3", notebook_path="wordslab-notebooks-lib/nbs/test.ipynb")

'55d9b34d-7e39-4cb6-9c95-eb0b9e0176f3'

In [ ]:
#| export
@patch
async def run_cell(
    self: WordslabNotebook,
    cell_id: str = None,  # id of cell to execute
):
    """"Adds the cell identified by `cell_id` to the run queue, only in the current notebook (jupyterlab 'run-cell' command limitation).
    Returns the cell id.
    DOES NOT return the result of the execution: the target cell will only be run after the current cell execution finishes.
    Use the `read_cell` method later with the same `cell_id` to get the result of the execution."""
    self._ensure_jupyterlab_extension()
    if not cell_id:
        raise ValueError("`cell_id` parameter is mandatory")
    result = await self._comm.send({'action': 'run_cell', 'cell_id': cell_id})
    if 'success' in result and result['success']:
        return result['cell_id']
    elif 'error' in result:
        raise RuntimeError(result['error'])

In [ ]:
code_cell_id = await notebook.add_cell(cell_type="code", content="1+1")

In [ ]:
1+1

2

In [ ]:
await notebook.run_cell(cell_id=code_cell_id)

'2a7bc090-ef99-4a21-bf9e-475d24de51c5'

In [ ]:
#|export
@patch
def read_cell(
    self: WordslabNotebook,
    cell_id: str = None,  # id of cell to delete
):
    """"Read the text content of the cell identified by `cell_id`, only in the current notebook.
    Returns the text content of the cell as a single multiline string."""
    self._ensure_jupyterlab_extension()
    if not cell_id:
        raise ValueError("`cell_id` parameter is mandatory")
    cell = next((c for c in self.content.cells if c.id == cell_id), None)
    if not cell:
        raise ValueError(f"Cell not found: {cell_id}")
    return cell.source

In [ ]:
above_cell_id = "df396dbb-8ecf-41df-b4cf-1f484dabb0fb"

In [ ]:
notebook.read_cell(above_cell_id)

'#|export\n@patch\ndef read_cell(\n    self: WordslabNotebook,\n    cell_id: str = None,  # id of cell to delete\n):\n    """"Read the text content of the cell identified by `cell_id`, only in the current notebook.\n    Returns the text content of the cell as a single multiline string."""\n    self._ensure_jupyterlab_extension()\n    if not cell_id:\n        raise ValueError("`cell_id` parameter is mandatory")\n    cell = next((c for c in self.content.cells if c.id == cell_id), None)\n    if not cell:\n        raise ValueError(f"Cell not found: {cell_id}")\n    return cell.source'

## Explore the notebook variables, functions and objects

In [ ]:
#| export
@patch
def show_variables_and_functions(self: WordslabNotebook):
    """Display the variables and functions defined by the user so far in the notebook."""
    variables, functions = get_ipython().user_items()
    output = "<h4>Variables</h4><table><tr><th>Name</th><th>Value</th></tr>"
    for var, value in variables.items():
        output += f"<tr><td>{var}</td><td>{escape(value)}</td></tr>"
    output += "</table>"
    output += "<h4>Functions</h4><table><tr><th>Name</th><th>Signature</th></tr>"
    for func, sig in functions.items():
        output += f"<tr><td>{func}</td><td>{escape(sig)}</td></tr>"
    output += "</table>"
    display(HTML(output))

In [ ]:
notebook.show_variables_and_functions()

Name,Value
Code,"functools.partial(<function ft at 0x7b36600a9580>, 'code', void_=False)"
Source,"functools.partial(<function ft at 0x7b36600a9580>, 'source', void_=True)"
shell,<ipykernel.zmqshell.ZMQInteractiveShell object at 0x7b3670078680>
user_items,None
variables,"{'Code': ""functools.partial(<function ft at 0x7b36600a9580>, 'code', void_=False)"", 'Source': ""functools.partial(<function ft at 0x7b36600a9580>, 'source', void_=True)"", 'shell': '<ipykernel.zmqshell.…"
functions,"{'_safe_str': '(obj, max_str_len=200)'}"
get_variables_values,None
get_tools_schemas_and_functions,None
notebook,<__main__.WordslabNotebook object at 0x7b3640f956a0>
add_cell,None


In [ ]:
#| export
def _safe_getattr(obj, name):
    try:
        return getattr(obj, name)
    except:
        return None

def _safe_attr_doc(obj, name, max_str_len=200):
    # Try to get the static attribute from the class
    try:
        class_attr = getattr_static(obj.__class__, name)
    except AttributeError:
        class_attr = None
    # 1. Property
    if isinstance(class_attr, property):
        attr_doc = getdoc(class_attr)
    # 2. classmethod / staticmethod
    elif isinstance(class_attr, (classmethod, staticmethod)):
        attr_doc = getdoc(class_attr.__func__)
    # 3. Function (instance method)
    elif isfunction(class_attr):
        attr_doc = getdoc(class_attr)
    # 4. Descriptor with __doc__ (other descriptors)
    elif hasattr(class_attr, "__doc__") and class_attr.__doc__:
        attr_doc = getdoc(class_attr)
    # 5. Fallback to instance attribute
    elif hasattr(obj, name):
        instance_attr = getattr(obj, name)
        if not type(instance_attr).__module__ == "builtins" and hasattr(instance_attr, "__doc__") and instance_attr.__doc__:
            attr_doc = getdoc(instance_attr)
        else:
            attr_doc = ""
    # Max length
    if attr_doc is None:
        attr_doc = ""
    elif len(attr_doc) > max_str_len:
        attr_doc = attr_doc[:max_str_len] + "…"
    return attr_doc

@patch
def show_object_members(self: WordslabNotebook, obj):
    """Display the attributes and methods of a given python object"""
    obj_class = obj.__class__
    output = (f"<h3>Object of type: {obj_class.__name__}</h3>") 
    output += f"<pre>{getdoc(obj)}</pre>"

    obj_members = [(name, _safe_getattr(obj, name)) for name in dir(obj) if not name.startswith("_")]

    output += "<h4>Attributes</h4>"
    output += "<table><tr><th>Name</th><th>Type</th><th>Value</th><th>Doc</th></tr>"
    for name, value in obj_members:
        if name.startswith("_"):
            continue
        # Skip callables (handled as methods below)
        if callable(value):
            continue
        attr_type = type(value).__name__
        attr_value = _safe_str(value)
        attr_doc = _safe_attr_doc(obj, name)
        output += f"<tr><td>{name}</td><td>{escape(attr_type)}</td><td>{escape(attr_value)}</td><td>{escape(attr_doc)}</td></tr>"
    output += "</table>"

    output += "<h4>Methods</h4>"
    output += "<table><tr><th>Name</th><th>Signatue</th><th>Type</th><th>Doc</th></tr>"
    for name, value in obj_members:
        if name.startswith("_"):
            continue
        # Skip attributes (handled above)
        if not callable(value):
            continue
        # Determine method type
        method_type = "instance method"
        if isfunction(value):
            # Defined on the class
            class_attr = getattr(obj_class, name, None)
            if isinstance(class_attr, classmethod):
                method_type = "class method"
            elif isinstance(class_attr, staticmethod):
                method_type = "static method"
        elif ismethod(value):
            method_type = "instance method"
        # Signature
        try:
            sig = str(signature(value))
        except (ValueError, TypeError):
            sig = "(...)"
        # Doc
        method_doc = _safe_attr_doc(obj, name)
        output += f"<tr><td>{name}</td><td>{escape(sig)}</td><td>{method_type}</td><td>{escape(method_doc)}</td></tr>"
    output += "</table>"

    display(HTML(output))

In [ ]:
notebook.show_object_members(notebook)

Name,Type,Value,Doc
cell_id,str,b602a192-b7e7-4976-a26d-4fa1854879b4,"Unique ID of the current notebook cell, useful to locate the current cell in the full notebook content"
chat_model,str,qwen3:30b,
chat_thinking,bool,True,
content,NotebookNode,"{'metadata': {'kernelspec': {'display_name': 'wordslab-notebooks-lib', 'language': 'python', 'name': 'wordslab-notebooks-lib'}, 'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3}, …",Full content of the notebook returned as a NotebookNode object from the nbformat library
jupyterlab_extension_installed,bool,True,
jupyterlab_extension_version,str,0.0.13,wordslab-notebooks-lib version number injected by the Jupyterlab frontend extension
path,str,wordslab-notebooks-lib/nbs/04_notebook.ipynb,Relative path of the notebook .ipynb file in the notebook workspace
Name,Signatue,Type,Doc
JupyterlabExtensionComm,"(target_name='wordslab_notebooks', timeout=2.0)",instance method,
add_cell,"(content: str, placement: str = 'add_after', cell_id: str = None, cell_type: str = 'note', notebook_path: str = '')",instance method,"Add a cell to the current notebook or any other opened notebook (`notebook_path`), at the start/end of the notebook or before/after any cell (`placement`and `cell_id`), with a `cell_type` (note|prompt…"


## Get $variable values and &tool schemas referenced in prompt cells

In [ ]:
#| export
@patch
def get_variables_values(self: WordslabNotebook, var_names: list):
    """Get a safe and serializable representation of variables values."""
    return get_ipython().get_variables_values(var_names=var_names)

@patch
def get_tools_schemas_and_functions(self: WordslabNotebook, func_names: list):
    """Get a json schema of functions which can be used as tools."""
    return get_ipython().get_tools_schemas_and_functions(func_names=func_names)

In [ ]:
notebook.get_variables_values(var_names=["variables", "functions"])

{'variables': {'Code': "functools.partial(<function ft>, 'code', void_=False)",
  'Source': "functools.partial(<function ft>, 'source', void_=True)",
  'shell': '<ipykernel.zmqshell.ZMQInteractiveShell object>',
  'user_items': 'None'},
 'functions': {'_safe_str': '(obj, max_str_len=200)'}}

In [ ]:
notebook.get_tools_schemas_and_functions(["example_sum"])

{'example_sum': ({'type': 'function',
   'function': {'name': 'example_sum',
    'description': 'Adds a + b.\n\nReturns:\n- type: integer',
    'parameters': {'type': 'object',
     'properties': {'a': {'type': 'integer',
       'description': 'First thing to sum'},
      'b': {'type': 'integer',
       'description': 'Second thing to sum',
       'default': 1}},
     'required': ['a']}}},
  <function __main__.example_sum(a: int, b: int = 1) -> int>)}

## Collect the notebook context for LLM calls

The notebook cells format is documented here:

https://nbformat.readthedocs.io/en/latest/format_description.html

Code cell outputs are a list, where each item has an output_type. The main types are:

- stream — stdout/stderr text (e.g., from print())

Has name (stdout/stderr) and text fields

- execute_result — the return value of the last expression

Has data dict with MIME types like text/plain, text/html, image/png

- display_data — from display() calls

Same data dict structure as execute_result

- error — exceptions

Has ename, evalue, and traceback fields

The tricky part is that execute_result and display_data can contain multiple representations of the same data (e.g., a pandas DataFrame might have both text/plain and text/html versions).

> Here is an example of "note" cell

```python
{'id': 'eb560f48-42a2-4573-bf12-b3edb40bff20',
 'cell_type': 'markdown',
 'source': 'Code cell outputs in nbformat are a list, where each item has an output_type. The main types are:\n\n- stream — stdout/stderr text (e.g., from print())\n\nHas name (stdout/stderr) and text fields\n\n- execute_result — the return value of the last expression\n\nHas data dict with MIME types like text/plain, text/html, image/png\n\n- display_data — from display() calls\n\nSame data dict structure as execute_result\n\n- error — exceptions\n\nHas ename, evalue, and traceback fields\n\nThe tricky part is that execute_result and display_data can contain multiple representations of the same data (e.g., a pandas DataFrame might have both text/plain and text/html versions).',
 'metadata': {}}
```

> Here is an example of "prompt" cell

```python
{'id': '3d5a241d-890c-46db-acf5-d92886f9a77d',
 'cell_type': 'code',
 'source': '# This is an example of prompt\nprint("and this is an example of answer")',
 'metadata': {'trusted': True,
  'wordslab_cell_type': 'prompt',
  'execution': {'iopub.status.busy': '2025-12-29T15:47:42.347364Z',
   'iopub.execute_input': '2025-12-29T15:47:42.347549Z',
   'iopub.status.idle': '2025-12-29T15:47:42.350815Z',
   'shell.execute_reply.started': '2025-12-29T15:47:42.347534Z',
   'shell.execute_reply': '2025-12-29T15:47:42.349884Z'}},
 'outputs': [{'name': 'stdout',
   'output_type': 'stream',
   'text': 'and this is an example of answer\n'}],
 'execution_count': 248}
```

> Here is an example of code cell

This code

```python
import sys
from IPython.display import display, HTML, Markdown

# stream (stdout)
print("This is stdout")

# stream (stderr)
print("This is stderr", file=sys.stderr)

# display_data (multiple formats)
display(HTML("<b>Bold HTML</b>"))
display(Markdown("**Bold Markdown**"))

# execute_result (last expression)
{"key": "value", "number": 42}
```

Produces these outputs

```python
[{'name': 'stdout', 'output_type': 'stream', 'text': 'This is stdout\n'},
 {'name': 'stderr', 'output_type': 'stream', 'text': 'This is stderr\n'},
 {'output_type': 'display_data',
  'data': {'text/plain': '<IPython.core.display.HTML object>',
   'text/html': '<b>Bold HTML</b>'},
  'metadata': {}},
 {'output_type': 'display_data',
  'data': {'text/plain': '<IPython.core.display.Markdown object>',
   'text/markdown': '**Bold Markdown**'},
  'metadata': {}},
 {'execution_count': 223,
  'output_type': 'execute_result',
  'data': {'text/plain': "{'key': 'value', 'number': 42}"},
  'metadata': {}},
 {'traceback': ['\x1b[31m---------------------------------------------------------------------------\x1b[39m',
   '\x1b[31mValueError\x1b[39m                                Traceback (most recent call last)',
   '\x1b[36mCell\x1b[39m\x1b[36m \x1b[39m\x1b[32mIn[224]\x1b[39m\x1b[32m, line 1\x1b[39m\n\x1b[32m----> \x1b[39m\x1b[32m1\x1b[39m \x1b[38;5;28;01mraise\x1b[39;00m \x1b[38;5;167;01mValueError\x1b[39;00m(\x1b[33m"\x1b[39m\x1b[33mExample error message\x1b[39m\x1b[33m"\x1b[39m)\n',
   '\x1b[31mValueError\x1b[39m: Example error message'],
  'ename': 'ValueError',
  'evalue': 'Example error message',
  'output_type': 'error'}]
```

In this code cell

```python
{'id': 'a1d9fbe2-9a84-4d7d-9415-a2e4693ba7ac',
 'cell_type': 'code',
 'source': 'import sys\nfrom IPython.display import display, HTML, Markdown\n\n# stream (stdout)\nprint("This is stdout")\n\n# stream (stderr)\nprint("This is stderr", file=sys.stderr)\n\n# display_data (multiple formats)\ndisplay(HTML("<b>Bold HTML</b>"))\ndisplay(Markdown("**Bold Markdown**"))\n\n# execute_result (last expression)\n{"key": "value", "number": 42}',
 'metadata': {'trusted': True,
  'execution': {'iopub.status.busy': '2025-12-29T15:07:25.670149Z',
   'iopub.execute_input': '2025-12-29T15:07:25.670496Z',
   'iopub.status.idle': '2025-12-29T15:07:25.678474Z',
   'shell.execute_reply.started': '2025-12-29T15:07:25.670471Z',
   'shell.execute_reply': '2025-12-29T15:07:25.677678Z'}},
 'outputs': [...],
 'execution_count': 223}
```

The following methods are inspired by the library **toolslm** from **Answer.ai**, but are adapted to our specific goal with prompt cells.

In [ ]:
#| export
def _mime_bundle_to_text(data):
    "Get text from MIME bundle, preferring markdown over plain"
    if 'text/markdown' in data:
        return ('markdown', ''.join(list(data['text/markdown'])))
    if 'text/html' in data:
        return ('html', ''.join(list(data['text/html'])))
    if 'text/plain' in data: 
        return ('text', ''.join(list(data['text/plain'])))

def _cell_output_to_xml(o):
    "Convert single notebook output to XML format"
    # execute_result — the return value of the last expression, or calls to display()
    if hasattr(o, 'data'):
        mime, txt = _mime_bundle_to_text(o.data)
        if txt:
            return Out(txt, type=mime)
    # stream — stdout/stderr text (e.g., from print())
    if hasattr(o, 'text'):
        txt = o.text if isinstance(o.text, str) else ''.join(o.text)
        return Out(txt, type=o.get('name', 'stdout'))
    # Error - exceptions
    if hasattr(o, 'ename'):
        return Out(f"{o.ename}: {o.evalue}", type='error')

Test

```python
[to_xml(_cell_output_to_xml(o)) for o in example_output]
```

Result

```python
['<out type="stdout">This is stdout\n</out>',
 '<out type="stderr">This is stderr\n</out>',
 '<out type="html">&lt;b&gt;Bold HTML&lt;/b&gt;</out>',
 '<out type="markdown">**Bold Markdown**</out>',
 '<out type="text">{\'key\': \'value\', \'number\': 42}</out>',
 '<out type="error">ValueError: Example error message</out>']
 ```

In [ ]:
#| export
def _cell_to_xml(cell):
    "Convert notebook cell to concise XML format"
    src = ''.join(getattr(cell, 'source', ''))
    if cell.cell_type == 'markdown':
        return Note(src)
    elif cell.cell_type == 'code':
        out_items = L(getattr(cell,'outputs',[])).map(_cell_output_to_xml).filter()
        is_prompt = "wordslab_cell_type" in cell.metadata and cell.metadata["wordslab_cell_type"] == "prompt"
        if is_prompt:
            parts = [User(src)]
            if out_items:
                parts.append(Assistant(*out_items))
            return Prompt(*parts)
        else:
            parts = [Source(src)]
            if out_items:
                parts.append(Outputs(*out_items))
            return Code(*parts)
    else:
        return None

Test

```python
to_xml(_cell_to_xml(note_cell))
```

Result

```python
'<note>Code cell outputs in nbformat are a list, where each item has an output_type. The main types are:\n\n- stream — stdout/stderr text (e.g., from print())\n\nHas name (stdout/stderr) and text fields\n\n- execute_result — the return value of the last expression\n\nHas data dict with MIME types like text/plain, text/html, image/png\n\n- display_data — from display() calls\n\nSame data dict structure as execute_result\n\n- error — exceptions\n\nHas ename, evalue, and traceback fields\n\nThe tricky part is that execute_result and display_data can contain multiple representations of the same data (e.g., a pandas DataFrame might have both text/plain and text/html versions).</note>'
```

Test

```python
to_xml(_cell_to_xml(prompt_cell))
```

Result

```python
'<prompt><user># This is an example of prompt\nprint("and this is an example of answer")</user><assistant><out type="stdout">and this is an example of answer\n</out></assistant></prompt>'

```

In [ ]:
#| export
def _cells_to_notebook_xml(cells):
    cells_xml = [_cell_to_xml(c) for c in cells if c.cell_type in ('code', 'markdown')]
    return "\n".join(L(cells_xml).map(partial(to_xml, do_escape=False)))

Test

```python
to_xml(_cells_to_notebook_xml([note_cell, prompt_cell, code_cell]))
```

Result

```python
'<note>Code cell outputs in nbformat are a list, where each item has an output_type. The main types are:\n\n- stream — stdout/stderr text (e.g., from print())\n\nHas name (stdout/stderr) and text fields\n\n- execute_result — the return value of the last expression\n\nHas data dict with MIME types like text/plain, text/html, image/png\n\n- display_data — from display() calls\n\nSame data dict structure as execute_result\n\n- error — exceptions\n\nHas ename, evalue, and traceback fields\n\nThe tricky part is that execute_result and display_data can contain multiple representations of the same data (e.g., a pandas DataFrame might have both text/plain and text/html versions).</note><prompt><user># This is an example of prompt\nprint("and this is an example of answer")</user><assistant><out type="stdout">and this is an example of answer\n</out></assistant></prompt><code><source>import sys\nfrom IPython.display import display, HTML, Markdown\n\n# stream (stdout)\nprint("This is stdout")\n\n# stream (stderr)\nprint("This is stderr", file=sys.stderr)\n\n# display_data (multiple formats)\ndisplay(HTML("&lt;b&gt;Bold HTML&lt;/b&gt;"))\ndisplay(Markdown("**Bold Markdown**"))\n\n# execute_result (last expression)\n{"key": "value", "number": 42}<outputs><out type="stdout">This is stdout\n</out><out type="stderr">This is stderr\n</out><out type="html">&lt;b&gt;Bold HTML&lt;/b&gt;</out><out type="markdown">**Bold Markdown**</out><out type="text">{\'key\': \'value\', \'number\': 42}</out></outputs></code>'
```

In [ ]:
#| export
@patch
def get_context_for_llm(self: WordslabNotebook):
    # Get current notebook state
    all_cells = self.content.cells
    current_cell_id = self.cell_id

    # Exclude all cells after the previous one
    previous_cells = all_cells[:next((i for i, c in enumerate(all_cells) if c.id == current_cell_id), len(all_cells))]

    # Then exclude all cells hidden from AI 
    context_cells = [cell for cell in previous_cells if "wordslab_hide_from_ai" not in cell.metadata]

    # Convert the remaining markdown and code cells to XML
    notebook_context_xml = _cells_to_notebook_xml(context_cells)

    # Return a XML string version of the notebook
    return to_xml(notebook_context_xml)

In [ ]:
# Estimated number of tokens for this notebook
int(len(notebook.get_context_for_llm())/3)

16843

## notebook.chat - the notebook assistant

### Prompt template

In [ ]:
#| export
prompt_template = """
<system_instructions>

<role_and_behavior>
You are an AI assistant designed to help the user learn and solve problems interactively.
You work with the user step-by-step rather than just giving complete answers. You are especially good at:
- Breaking down complex topics into manageable pieces
- Helping the user work through coding problems in Python
- Encouraging the user to try things himself, with guidance when he needs it
- Adapting to the user level and interests
You are designed to be collaborative - you ask questions, check the user understanding, and let him explore ideas rather than just lecturing. 
You can help with teaching, coding, problem-solving, research, and creative projects. You assume that the user is very smart.
What sets you apart is your teaching approach - you focus on helping the user develop his skills rather than just giving him answers. 
You provide information in small chunks, check in frequently to see if things make sense, and encourage the user to try things himself.
</role_and_behavior>

<execution_context_info>
Your design is heavily inspired by the Solveit platform developed by Answer.ai, but adapted for a local and open source Jupyterlab environment.
You run in an interactive Juyter notebook environment where the user can take notes, write code, and chat with you.
This notebook mixes three types of cells:
- "note" cells (green border), which contain markdown text written by the user
- "prompt" cells (red border), which contain a user instruction and an assistant answer, both in markdown text
- "code" cells (blue border), which contain python source code and optionnaly outputs - the result of the execution
The cells are meant to be read and executed in chronological order from top to bottom.
You receive this instruction in the following context :
- the user just executed a "prompt" cell
- in this cell, he typed a user instruction which will be provided below
- you must execute this user instruction TAKING INTO ACCOUNT all the previous cells in the notebook
- the content of the previous cells is also provided below, you must interpret it as a CONVERSATION HISTORY
- your answer will be rendered as an output of the prompt cell in markdown format
The "code" cells of the notebook are backed by a python kernel which maintains state with functions and variables. 
The user can optionnaly provide a set of python functions for you to use as tools:
- if the user mentions the function with this very specific syntax `&myfunc` somewhere in the notebook (backticks mandatory)
- then the description of the function myfunc and its parameters will be provided to you as a tool you can call
Be careful to always check first if you could call these tools to better ground your answer, instead of trying to guess based on your pretraining knwoledge.
Only if you were not provided with the right tool, you can also generate and display code blocks in your response, that the user will be able to review and copy in a new "code" cell to execute it.
The user can optionnaly provide some of the python variables values as information for you:
- if the user mentions the variable with this very specific syntax `$myvar` somewhere in the notebook (backticks mandatory)
- then the value of the variable myvar will be provided to you as information to use to execute the user instruction
Same remark: prioritize using this information to ground your answer to the user question instead of trying to guess.
</execution_context_info>

<prompt_format_spec>
According to this execution context information, the prompt will be structured with XML tags as follows
- conversation_history -> list of cells of the notebook, which represents the history of the conversation
  - note -> markdown cell, directly contains a note from the user
  - prompt -> prompt sent to you during a previous turn of the conversation
    - user -> instruction written by the user
    - assistant -> answer previously generated by you
      - out -> part of the answer which was displayed
  - code -> python cell executed by the user
    - source -> python code written and executed by the user
    - outputs -> results of the python kernel execution
      - out -> part of the execution result which was displayed     
- referenced_variables -> optional
  - var name=... -> value of python variable referenced by the user (may be truncated if too long)
- user_instruction -> the very last user instruction written in a prompt cell you need to execute 
The definitions of the tools you can call are injected through the API.
</prompt_format_spec>

<output_format_spec>
Make sure you always give the FINAL answer in the same language as the user instruction.
For example, if the user_instruction below is written in french, answer in french, if it is written in german, answer in german.
But off course, for the intermediate turns in an agentic loop, this rule doesn't apply: you can generate as many tool calls as needed before generating the FINAL answer.
Try to be concise: just provide the answer, don't explain the obvious unless explicity prompted to do so.
Assume the user is intelligent and has no time to waste reading too long answers.
When you generate code, make sure to do it inside markdown fenced blocks.
</system_instructions>

<conversation_history>

{notebook_context}

</conversation_history>

<referenced_variables>

{referenced_variables}

</referenced_variables>

<user_instruction>

{user_instruction}

</user_instruction>
"""

Syntax to reference tools and variables

In [ ]:
#| export
FUNC_RE = re.compile(r"`&([a-zA-Z_][a-zA-Z0-9_]*)`")
VAR_RE  = re.compile(r"`\$([a-zA-Z_][a-zA-Z0-9_]*)`")

### Tools and variables

Define test tools and variables

In [ ]:
def add(a: int,  # The first number
        b: int   # The second number
       ) -> int: # The sum of the two numbers
  """Add two numbers"""
  return a + b


def multiply(a: int,  # The first number 
             b: int   # The second number
            ) -> int: # The product of the two numbers
  """Multiply two numbers"""
  return a * b

cat_name = "My cat is named Jerry"
dog_name = "My dog is named Rex"

Mention them so they are available to the AI assistant:

- you can use functions `&add`, `&multiply` as tools
- you can use variables `$cat_name`, `$dog_name`as variables

In [ ]:
context = notebook.get_context_for_llm()

In [ ]:
funcs_names = FUNC_RE.findall(context)
vars_names = VAR_RE.findall(context)

print(funcs_names)
print(vars_names)

['myfunc', 'add', 'multiply']
['myvar', 'cat_name', 'dog_name']


In [ ]:
tools_schemas_and_functions = notebook.get_tools_schemas_and_functions(funcs_names)
tools_schemas_and_functions

{'add': ({'type': 'function',
   'function': {'name': 'add',
    'description': 'Add two numbers\n\nReturns:\n- type: integer',
    'parameters': {'type': 'object',
     'properties': {'a': {'type': 'integer',
       'description': 'The first number'},
      'b': {'type': 'integer', 'description': 'The second number'}},
     'required': ['a', 'b']}}},
  <function __main__.add(a: int, b: int) -> int>),
 'multiply': ({'type': 'function',
   'function': {'name': 'multiply',
    'description': 'Multiply two numbers\n\nReturns:\n- type: integer',
    'parameters': {'type': 'object',
     'properties': {'a': {'type': 'integer',
       'description': 'The first number'},
      'b': {'type': 'integer', 'description': 'The second number'}},
     'required': ['a', 'b']}}},
  <function __main__.multiply(a: int, b: int) -> int>)}

In [ ]:
tools = Tools([t[1] for t in tools_schemas_and_functions.values()])
tools.get_functions()

[<function __main__.add(a: int, b: int) -> int>,
 <function __main__.multiply(a: int, b: int) -> int>]

In [ ]:
var_values = notebook.get_variables_values(vars_names)
var_values

{'cat_name': 'My cat is named Jerry', 'dog_name': 'My dog is named Rex'}

In [ ]:
referenced_variables = "\n".join(L([Var(value, name=name) for name,value in var_values.items()]).map(to_xml))
referenced_variables

'<var name="cat_name">My cat is named Jerry</var>\n<var name="dog_name">My dog is named Rex</var>'

In [ ]:
#| export
@patch
def set_ollama_chat_model(self: WordslabNotebook, 
                          model: str,
                          think: Union[bool, Literal["low", "medium", "high"], None] = None,
                          context_size: int = 32768, # This is the default value for the ollama server in wordslab-notebooks
                          web_search: bool = False, # The ollama API key is necessary to activate web search
                          base_url: str = "http://localhost:11434",
                          api_key: Optional[str] = None,  # If not provided, the optional key will be pulled from WordslabEnv
                          ): 
    self.chat_model_client = OllamaModelClient(model, context_size=context_size, base_url=base_url, api_key=api_key)
    self.chat_model = model
    self.chat_model_think = think
    self.chat_model_context_size = context_size
    self.chat_model_web_search = web_search

In [ ]:
#| export
@patch
def set_openrouter_chat_model(self: WordslabNotebook, 
                          model: str,
                          think: Union[bool, Literal["xhigh", "high", "medium", "low", "minimal", "none"], int, None] = None,
                          context_size: Optional[int] = None, # For OpenRouter this parameter is ignored, we inherit the remote model config
                          web_search: bool = True, # Web search is activated by default four cloud models in openrouter
                          base_url: str = "https://openrouter.ai/api/v1",
                          api_key: Optional[str] = None, # If not provided, the mandatory key will be pulled from WordslabEnv
                          ): 
    self.chat_model_client = OpenRouterModelClient(model, context_size=context_size, base_url=base_url, api_key=api_key)
    self.chat_model = model
    self.chat_model_think = think
    self.chat_model_context_size = context_size
    self.chat_model_web_search = web_search

In [ ]:
#| export
@patch
def chat(self: WordslabNotebook, user_instruction: str):    
    # Ensure the model client is initialized
    if not hasattr(self,"chat_model_client"):
        env = WordslabEnv()
        agent_model = env.default_model_agent
        context_length = env.default_model_context_length
        self.set_ollama_chat_model(agent_model, think=True, context_size=context_length)
    
    # Get notebook cont.ext
    notebook_context = self.get_context_for_llm()
    # Extract referenced tools and variables
    funcs_names = FUNC_RE.findall(notebook_context)
    vars_names = VAR_RE.findall(notebook_context)
    # Get tools schemas 
    tools_schemas_and_functions = self.get_tools_schemas_and_functions(funcs_names)
    tools = Tools([t[1] for t in tools_schemas_and_functions.values()])
    # Get variables values
    vars_values = self.get_variables_values(vars_names)
    referenced_variables = "\n".join(L([Var(value, name=name) for name,value in vars_values.items()]).map(to_xml))
    
    # Format the prompt
    prompt = prompt_template.format(notebook_context=notebook_context,
                                    referenced_variables=referenced_variables,
                                    user_instruction=user_instruction)

    # Model agentic loop
    self.chat_model_client(user_prompt=prompt, think=self.chat_model_think, tools=tools, web_search=self.chat_model_web_search)

In [ ]:
import importlib
import wordslab_notebooks_lib.chat
importlib.reload(wordslab_notebooks_lib.chat)
from wordslab_notebooks_lib.chat import OllamaModelClient, OpenRouterModelClient, Tools

In [ ]:
notebook = WordslabNotebook()

In [ ]:
notebook.chat("Using only the provided tools to make no mistake, what is (11545468+78782431)*418742?")

> [Thinking] ... thought in 1285 words

> [Tool call] ... `add` returned `90327899`

> [Tool call] ... `multiply` returned `37824085083058`

> [Thinking] ... thought in 191 words

37824085083058



In [ ]:
def get_weather(
    city: str # A city name
) -> str: # A sentence describing the weather
    "A service predicting the weather city by city"
    return f"The weather is nice in {city} today"

You can use the service `&get_weather`.

In [ ]:
notebook.chat("What will the weather be like tomorrow in Paris?")

> [Thinking] ... thought in 309 words

> [Tool call] ... `get_weather` returned `The weather is nice in Paris today`

> [Thinking] ... thought in 199 words

The weather is nice in Paris today.



In [ ]:
notebook.chat("What's the name of my dog ?")

> [Thinking] ... thought in 272 words

Rex



Here is the check that the frontend extension will do before executing notebook.chat:

In [ ]:
("notebook" in globals()) and ("WordslabNotebook" in str(type(notebook)))

True

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

## Develop a Jupyterlab frontend extension

### Understand Jupyterlab kernels and frontend extensions

Jupyter kernels technical implementation details

https://chatgpt.com/share/692bea08-4510-8004-b9ab-c02feeb97c08

Jupyterlab extension development tutorial

https://jupyterlab.readthedocs.io/en/latest/extension/extension_tutorial.html

### Intialize the components of a frontend extension

The source code of the Jupyterlab frontend extension can be found in the following files:

Typescript source code, dependencies, and compilation config:

- `src/index.ts`
- `package.json`
- `tsconfig.json`
- `.yarnrc.yml`

Extension manifest and Javascript compiled code

- wordslab_notebooks_lib/labextension
  - package.json
  - static/remoteEntry.97d57e417eaf8ebadeb6.js 

This is how the extension files are included in the python package:

- `MANIFEST.in` 

```
include install.json
include package.json
recursive-include wordslab_notebooks_lib/labextension *

graft wordslab_notebooks_lib/labextension
graft src
```

This is how the extension files are installed in Jupyterlab extensions directory when the python package is installed:

- `pyproject.toml`

```toml
[tool.setuptools]
include-package-data = true 

[tool.setuptools.data-files]
"share/jupyter/labextensions/wordslab-notebooks-lib" = [
  "wordslab_notebooks_lib/labextension/package.json",
  "install.json"
]
"share/jupyter/labextensions/wordslab-notebooks-lib/static" = [
  "wordslab_notebooks_lib/labextension/static/*"
]
```

This how the command `jupyter labextension develop` finds the directory where the extension files live:

- `wordslab_notebooks_lib\__init__.py`

```python
def _jupyter_labextension_paths():
    return [{
        "src": "labextension",
        "dest": "wordslab-notebooks-lib"
    }]
```

This is how the python package is identified as a Jupyterlab extension in pypi:

- `pyproject.toml`

```
classifiers = [ "Framework :: Jupyter :: JupyterLab :: Extensions :: Prebuilt" ]
```

### Install the Jupyterlab frontend extension in development mode

Open a Terminal

```bash
cd $WORDSLAB_WORKSPACE/wordslab-notebooks-lib
source $JUPYTERLAB_ENV/.venv/bin/activate

# Install Javascript dependencies
jlpm install

# Build TypeScript extension
jlpm build

# Register the extension with JupyterLab during development
# jupyter labextension develop . --overwrite
rm $JUPYTERLAB_ENV/.venv/share/jupyter/labextensions/wordslab-notebooks-lib
ln -s $WORDSLAB_WORKSPACE/wordslab-notebooks-lib/wordslab_notebooks_lib/labextension/ $JUPYTERLAB_ENV/.venv/share/jupyter/labextensions/wordslab-notebooks-lib

# Verify extension is found
jupyter labextension list
```

### Test the Jupyterlab frontend extension 

After installing the extension in development mode once, you can iterate fast:
- update the code in `src/index.ts`
- build the extension with `jlpm build`

```bash
cd $WORDSLAB_WORKSPACE/wordslab-notebooks-lib
source $JUPYTERLAB_ENV/.venv/bin/activate

# Build TypeScript extension
jlpm build
```
- **refresh** the Jupyterlab single page app in your browser
- test the updated extension

No need to reinstall the extension or to restart Jupyterlab itself, just refresh your browser page.

## Explore the notebook format

https://nbformat.readthedocs.io/en/latest/format_description.html

In [ ]:
nb = nbformat.from_dict(__notebook_content)

code_language = nb.metadata.language_info.name
print("> " + code_language + " notebook")

for cell in nb.cells:
    if cell.id == __cell_id: break
        
    is_markdown = cell.cell_type == "markdown"
    is_code = cell.cell_type == "code"
    is_raw = cell.cell_type == "raw"

    print("---------------------")
    print("cell", cell.id, cell.cell_type)
    print("---------------------")
    if is_markdown:
        print(cell.source[:100])
    elif is_code:
        print(f"```{code_language}\n" + cell.source[:100] + "\n```")
    elif is_raw:
        print(cell.source[:100])
    if is_code and cell.execution_count>0 and len(cell.outputs)>0:
        print("---------------------")
        print("cell outputs", cell.id, cell.execution_count)
        print("---------------------")
        for output in cell.outputs:
            if output.output_type == "stream":
                print(f"<{output.name}>")
                print(output.text[:100])
                print(f"</{output.name}>")
            elif output.output_type == "display_data":
                print("<display>")
                if "data" in output:
                    print("  <data>")
                    repr(output.data)
                    print("  </data>")
                if "metadata" in output and len(output.metadata)>0:
                    print("  <metadata>")
                    repr(output.metadata)
                    print("  </metadata>")
                print("</display>")
            elif output.output_type == "execute_result":
                print("<result>")
                if "data" in output:
                    print("  <data>")
                    print(output.data)
                    print("  </data>")
                if "metadata" in output and len(output.metadata)>0:
                    print("  <metadata>")
                    print(output.metadata)
                    print("  </metadata>")
                print("</result>")
            elif output.output_type == "error":
                print("<error>")
                print(output.ename)
                print(output.evalue)
                for frame in output.traceback:
                    print(frame)
                print("</error>")
        print("---------------------")

> python notebook
---------------------
cell 9d8a6aa0-8f58-4860-bcc1-2bfbdcb438b6 markdown
---------------------
# wordslab-notebooks-lib.jupyterlab

> Access wordslab-notebooks Jupyterlab extension version, curre
---------------------
cell 68f3493d-c252-4eb4-844b-abbd68ed3a70 markdown
---------------------
## Work together with AI in a Jupyterlab notebook - the Solveit method

A Jupyter notebook is a conv
---------------------
cell 0ff6fbdc-4a54-4e29-acbb-07529df8cfdd markdown
---------------------
## Install the Jupyterlab extension - wordslab-notebooks-lib

If you want to use "prompt" cells, you
---------------------
cell 65cd4cf8-d77b-4026-b428-bbd9550ea971 markdown
---------------------
## Communicate with the Jupyterlab extension
---------------------
cell ece4d545-8f78-4232-82fb-e837ea0185e4 code
---------------------
```python
#| export
import nbformat
```
---------------------
cell af2f3f45-f0da-4d37-9e39-b37d19ba5650 code
---------------------
```python
class JupyterlabNotebo